# **Web Server Assigment**

# 1




## Overview
In this assignment, you will explore the fundamentals of socket programming for TCP connections in Python. You will learn how to create a socket, bind it to a specific address and port, and send and receive HTTP packets. Additionally, you will gain an understanding of the basic structure of HTTP header formats.

Your task is to develop a web server that processes one HTTP request at a time. The web server should:

1. Accept and parse the incoming HTTP request.
2. Retrieve the requested file from the server’s file system.
3. Construct an HTTP response message comprising the requested file preceded by appropriate header lines.
4. Send the response directly to the client.

If the requested file is not found on the server, the web server must return an HTTP "404 Not Found" message to the client.

## Code
Below is a skeleton code for the web server. Your task is to complete the code. The sections where you need to add your implementation are marked with `#-----#`. Some sections may require multiple lines of code.

## Running the Server
1. Place an HTML file (e.g., `HelloWorld.html`) in the same directory as the server program.
2. Execute the server program.
3. Determine the IP address of the machine running the server (e.g., `128.238.251.26`).
4. From a different machine or the same machine, open a web browser and navigate to the URL corresponding to the server. For example:
   ```
   http://128.238.251.26:6789/HelloWorld.html
   ```
   In this example, `HelloWorld.html` is the name of the HTML file you placed in the server directory. Note the port number `6789` used after the colon. Replace this port number with the one configured in your server code.

5. The browser should display the contents of `HelloWorld.html`. If the port number is omitted (e.g., `http://128.238.251.26/HelloWorld.html`), the browser will default to port 80. In this case, ensure your server is listening on port 80.
6. Test requesting a file that is not present on the server. You should receive a "404 Not Found" message in the browser.


## What to Submit
Submit the following items:

1. The complete server code.
2. Screenshots of your client browser demonstrating:
   - Successful retrieval of the HTML file content from the server.
   - Receiving a "404 Not Found" message for a non-existent file.

---

In [ ]:
#import socket module
from socket import *
import sys  # In order to terminate the program

serverPort = 12000
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(1)  # Use an integer instead of 'True' for max queued connections

print("Ready to serve...")

while True:
    # Establish the connection
    print("Ready to serve...")
    connectionSocket, addr = serverSocket.accept()
    print(f"Connection received from: {addr}")

    try:
        # Receive the request message from the client
        message = connectionSocket.recv(1024).decode()  # FIXED: Added parentheses to decode()
        print(message)

        # Extract the requested filename from the GET request
        filename = message.split()[1]  # Extracts something like "/HelloWorld.html"
        filepath = filename[1:]  # Remove the leading "/"

        # Open the requested file
        with open(filepath, "r") as f:
            outputdata = f.read()

        # Send HTTP response headers
        connectionSocket.send("HTTP/1.1 200 OK\r\n".encode())
        connectionSocket.send("Content-Type: text/html\r\n\r\n".encode())

        # Send the content of the requested file
        connectionSocket.send(outputdata.encode())

    except FileNotFoundError:
        # Send response message for file not found (404)
        error_message = "<html><body><h1>404 Not Found</h1></body></html>"
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n".encode())
        connectionSocket.send("Content-Type: text/html\r\n\r\n".encode())
        connectionSocket.send(error_message.encode())

    finally:
        # Close the client connection
        connectionSocket.close()

serverSocket.close()
sys.exit()  # Terminate the program


Ready to serve...
Ready to serve...
Connection received from: ('172.23.42.155', 54899)
GET /HelloWorld.html HTTP/1.1
Host: 172.23.42.155:12000
Connection: keep-alive
Cache-Control: max-age=0
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate
Accept-Language: en-US,en;q=0.9,en-IN;q=0.8


Ready to serve...
Connection received from: ('172.23.42.155', 54900)
GET /HelloWorld.html HTTP/1.1
Host: 172.23.42.155:12000
Connection: keep-alive
Cache-Control: max-age=0
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/we

# 2

Currently, the web server handles only one HTTP request at a time. Implement a multithreaded server
that is capable of serving multiple requests simultaneously. Using threading, first create a main thread
in which your modified server listens for clients at a fixed port. When it receives a TCP connection
request from a client, it will set up the TCP connection through another port and services the client
request in a separate thread. There will be a separate TCP connection in a separate thread for each
request/response pair.

In [ ]:
import threading
import socket
import sys

# Server configuration
serverPort = 12000

# Create a server socket (TCP)
serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(5)  # Allow up to 5 pending connections

print(f"Server is ready to serve on port {serverPort}...")

# Function to handle client requests
def handle_client(connectionSocket, addr):
    try:
        message = connectionSocket.recv(1024).decode()
        
        if not message:
            print(f"Empty request from {addr}, closing connection.")
            connectionSocket.close()
            return

        print(f"Received request from {addr}: {message}")

        # Extract requested filename
        request_parts = message.split()
        if len(request_parts) < 2:
            print(f"Malformed request from {addr}")
            connectionSocket.send("HTTP/1.1 400 Bad Request\r\n\r\n".encode())
            connectionSocket.close()
            return

        filename = request_parts[1][1:]  # Remove leading "/"

        # Open the requested file
        with open(filename, "r") as f:
            outputdata = f.read()

        # Send HTTP response header
        connectionSocket.send("HTTP/1.1 200 OK\r\n".encode())
        connectionSocket.send("Content-Type: text/html\r\n\r\n".encode())

        # Send file content
        for i in outputdata:
            connectionSocket.send(i.encode())

        connectionSocket.send("\r\n".encode())

    except FileNotFoundError:
        # Handle file not found
        print(f"File not found: {filename}")
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
        connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>\r\n".encode())

    except Exception as e:
        print(f"Error handling request from {addr}: {e}")

    finally:
        connectionSocket.close()

# Main server loop
while True:
    try:
        connectionSocket, addr = serverSocket.accept()
        print(f"New connection from {addr}")

        # Create a new thread to handle the client request
        client_thread = threading.Thread(target=handle_client, args=(connectionSocket, addr))
        client_thread.start()

    except KeyboardInterrupt:
        print("\nShutting down the server...")
        serverSocket.close()
        sys.exit()


Server is ready to serve on port 12000...
New connection from ('172.23.42.155', 55120)
Received request from ('172.23.42.155', 55120): GET /HelloWorld.html HTTP/1.1
Host: 172.23.42.155:12000
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate
Accept-Language: en-US,en;q=0.9,en-IN;q=0.8


New connection from ('172.23.42.155', 55121)
New connection from ('172.23.42.155', 55122)
Received request from ('172.23.42.155', 55121): GET /HelloWorld.html HTTP/1.1
Host: 172.23.42.155:12000
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0
Accept: text/h

Main thread

Creates a socket and listens for incoming connections.
When a client connects, it spawns a new thread to handle the request.
Worker thread (handle_client)

Receives the HTTP request.
Extracts the requested file and serves it if found.
If the file is missing, returns a 404 Not Found message.
Closes the connection after processing.

# 3

The Multi Thread Web Server is now set and working but cannot handle changes in the source files. How can we look for changes in the source directory and access it in your code ? Explore Libraries in python

The watchdog observer monitors the directory for file changes.
Whenever a file is modified, on_modified() prints a message.
The handle_client function always reads the file from disk, so the latest content is served without restarting the server.

After you've accessed it in your code, Post a notification on the hosted http page to reload if any source directory changes

To notify clients about changes in the source directory and prompt them to reload, we can implement the following strategy:

(1)Global Change Tracking: Use a global flag or counter to track changes in the source directory. When a file is modified, the counter or flag will update.

(2)Embed Notification Script: Use a simple HTML page with a JavaScript snippet to poll the server for changes. When the server detects a change, it will respond to the client, prompting a reload.

(3)Integration with watchdog: Update the global change counter or flag whenever a file is modified or created.

Your dynamic http server is now ready